In [22]:
import robotic as ry
import numpy as np
from visual import getObject, lookAtObj, scanObject
from arena import *
import open3d as o3d
import random
import copy

INITIAL_OBJ_POS = [-.5, 0, .69]
ON_REAL = False
ARENA_POS = np.array([-.54, -.17, .651])

GRIPPER_FINGER_SEPARATION = .075
GRIPPER_FINGER_WIDTH = .025

def ICP_matrix(source, target, iterations=10, threshold=.02, trans_init=np.identity(4)):
    source_copy = copy.deepcopy(source)

    cumulative_transformation = np.identity(4)

    for _ in range(iterations):
        source_filtered, _ = source_copy.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        target_filtered, _ = target.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        source_downsampled = source_filtered.voxel_down_sample(voxel_size=0.001)
        target_downsampled = target_filtered.voxel_down_sample(voxel_size=0.001)

        reg_p2p = o3d.pipelines.registration.registration_icp(
            source_downsampled, target_downsampled, threshold, trans_init,
            o3d.pipelines.registration.TransformationEstimationPointToPoint())
        cumulative_transformation = np.dot(reg_p2p.transformation, cumulative_transformation)
        source_copy.transform(reg_p2p.transformation)

    return cumulative_transformation

def mstorePCR(point_clouds, smoothing=False, verbose=0):
    result = point_clouds[0]
    
    for i in range(len(point_clouds)-1):
        source, target = point_clouds[i+1], point_clouds[i]
        m = ICP_matrix(source, target)

        source.transform(m)
        result += source

        if smoothing:
            result, _ = result.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
            result = result.voxel_down_sample(voxel_size=0.001)

    if verbose:
        o3d.visualization.draw_geometries([result])
    return result

In [23]:
#-- load parameters, typically automatically from 'rai.cfg'
ry.params_print()
verbose = 0

#-- define a configuration
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

obj = C.addFrame('obj')
obj.setShape(ry.ST.ssBox, size=[.05, .09, .1, .005])
obj.setPosition(INITIAL_OBJ_POS)
obj.setColor([1, .5, .5])
obj.setMass(.1)
obj.setContact(True)

C.addFrame('predicted_obj') \
    .setShape(ry.ST.marker, size=[.1]) \
    .setPosition(INITIAL_OBJ_POS) \
    .setColor([1, 0, 0])

for i in range(32):
    C.addFrame(f'view_point_{i}') \
        .setPosition([0, 0, 0]) \
        .setShape(ry.ST.sphere, size=[.02]) \
        .setColor([1, 1, 0])

bot = ry.BotOp(C, ON_REAL)

obj_pos = INITIAL_OBJ_POS

# Generate Arena
arena = CircularArena(middleP=ARENA_POS, innerR=None, outerR=.3)
arena.plotArena(C)

-- module.cpp:operator():95(0) python,
RealSense/resolution: 640,
RealSense/alignToDepth!,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/multiBody!,
bot/useGripper,
bot/useRobotiq!,
bot/useArm: left,
bot/blockRealRobot!,
botsim/hyperSpeed: 1,
botsim/verbose: 1,
botsim/engine: physx,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
bot/useOptitrack!,
bot/useAudio!,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/useFCL,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopIters: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
opt/muInit: 1,
opt/m

In [24]:
# Point towards set initial object position
lookAtObj(bot, C, np.array(INITIAL_OBJ_POS))

# Capture midpoint from point cloud
obj_pos  = getObject(bot, C, arena) 

scanned_views_ = scanObject(bot, C, np.array(obj_pos), arena, view_count=8)
scanned_views = []
for sv in scanned_views_:
    point_cloud_obj = o3d.geometry.PointCloud()
    point_cloud_obj.points = o3d.utility.Vector3dVector(sv)
    scanned_views.append(point_cloud_obj)

{ time: 0.001341, evals: 10, done: 1, feasible: 1, sos: 0.0102844, f: 0, ineq: 0, eq: 0.0113478 }
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
{ time: 0.004614, evals: 19, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00303901 }
{ time: 0.001336, evals: 13, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0037375 }
{ time: 0.012593, evals: 12, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00428433 }
{ time: 0.025676, evals: 65, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0721808 }
{ time: 0.001169, evals: 11, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00361221 }
{ time: 0.002629, evals: 20, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00308265 }
{ time: 0.030974, evals: 352, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.825349 }
{ time: 0.006385, evals: 39, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.822365 }


In [25]:
final_points = mstorePCR(scanned_views)

In [26]:
pclFrame = C.getFrame("pcl")
pclFrame.setPointCloud(np.asarray(final_points.points))
C.view_recopyMeshes()

In [27]:

def validSideDistance(p1, p2, n):
    dist = np.linalg.norm(p2-p1)
    normal_dist = abs(np.dot(p1 - p2, n) / np.linalg.norm(n))
    side_dist = np.sqrt(dist*dist-normal_dist*normal_dist)
    return side_dist <= GRIPPER_FINGER_WIDTH*.5

def validNormalDistance(p1, p2, n, dist_thresh=GRIPPER_FINGER_SEPARATION):
    return (abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) <= dist_thresh
            and abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) >= .025)

def validNormalRelativeDir(p1, p2, n):
    p2pvec = p2-p1
    dot_product = np.dot(n, p2pvec)
    # Normal vector has magnitude one so we ignore it
    magnitude_vector = np.linalg.norm(p2pvec)
    cosine_angle = dot_product / magnitude_vector
    return cosine_angle <= 0

def validNormalDir(n1, n2):
    upvec = np.array([0., 0., 1.])
    dot_product1 = np.dot(n1, upvec)
    dot_product2 = np.dot(n2, upvec)
    return dot_product1 >= 0 and dot_product2 >= 0

def inPatches(pcd, pIndex, equal_normal_thresh=.7, planar_distance_thresh=.002, dist=GRIPPER_FINGER_WIDTH*2/3):
    pcd_points = np.asarray(pcd.points)
    point = pcd_points[pIndex]

    distances = np.linalg.norm(pcd_points - point, axis=1)

    within_distance_indices = np.where(distances <= dist)[0]

    pNormal = np.asarray(pcd.normals[pIndex])

    on_plane = []
    D = -(np.dot(pNormal, point))
    for i in within_distance_indices:
        plane_distance = np.abs(np.dot(pNormal, pcd_points[i]) + D) / np.linalg.norm(pNormal)
        if plane_distance <= planar_distance_thresh:
            on_plane.append(i)

    patch_points = []
    for i in on_plane:
        comp = np.asarray(pcd.normals[i])
        if np.dot(pNormal, comp) > equal_normal_thresh:
            patch_points.append(i)

    return patch_points

def calculate_valid_antipodal_pairs(pcd, min_patch_points=15, equal_normal_thresh=.9, verbose=0):

    # Prepare point cloud
    pcd = pcd.voxel_down_sample(voxel_size=.005)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.025, max_nn=20))

    if verbose:
        print(f"Finding possible grasp positions for point cloud with {len(pcd.points)} points.")

    # Find points in patches
    points_in_patches = []
    for i in range(len(pcd.points)):
        patch = inPatches(pcd, i, equal_normal_thresh=equal_normal_thresh)
        if len(patch) >= min_patch_points:
            points_in_patches.append(i)
            
            if verbose > 2:
                patch = np.array(patch)
                num_points = np.asarray(pcd.points).shape[0]
                colors = np.tile([0, 0, 1], (num_points, 1))
                colors[patch] = [1, 0, 0]
                pcd.colors = o3d.utility.Vector3dVector(colors)
                o3d.visualization.draw_geometries([pcd], point_show_normal=True)


    # Find antipodal pairs
    antipodal_pairs = []
    for i in range(len(points_in_patches)):
        p1 = points_in_patches[i]
        normal_i = np.asarray(pcd.normals[p1])
        for j in range(i + 1, len(points_in_patches)):
            p2 = points_in_patches[j]
            normal_j = np.asarray(pcd.normals[p2])
            dot_product = np.dot(normal_i, normal_j)

            if ((dot_product < -equal_normal_thresh or dot_product > equal_normal_thresh) and
                validSideDistance(pcd.points[p1], pcd.points[p2], pcd.normals[p1]) and
                validNormalDistance(pcd.points[p1], pcd.points[p2], pcd.normals[p1]) and
                validNormalRelativeDir(pcd.points[p1], pcd.points[p2], pcd.normals[p1])
                and validNormalDir(pcd.points[p2], pcd.normals[p1])
                ):
                    antipodal_pairs.append((p1, p2))
        
        if verbose and (i+1)%100 == 0:
            print(f"Searching Possible Grasps... (Searched {((i+1)/len(pcd.points)*100):.2f}, Total Points: {len(pcd.points)})")


    if verbose:
        print("Antipodal pairs found:", antipodal_pairs)

    if verbose > 1:
        line_set = o3d.geometry.LineSet()

        lines = []
        for pair in antipodal_pairs:
            lines.append([pair[0], pair[1]])
        lines = np.array(lines).astype(np.int32)
        line_set.points = o3d.utility.Vector3dVector(np.asarray(pcd.points))
        line_set.lines = o3d.utility.Vector2iVector(lines)

        line_colors = np.array([[1, 0, 0] for _ in range(len(lines))])
        line_set.colors = o3d.utility.Vector3dVector(line_colors)

        o3d.visualization.draw_geometries([pcd, line_set], point_show_normal=True)

    return antipodal_pairs

point_indices = calculate_valid_antipodal_pairs(final_points, verbose=2)

Finding possible grasp positions for point cloud with 1947 points.
Searching Possible Grasps... (Searched 5.14, Total Points: 1947)
Searching Possible Grasps... (Searched 10.27, Total Points: 1947)
Searching Possible Grasps... (Searched 15.41, Total Points: 1947)
Searching Possible Grasps... (Searched 20.54, Total Points: 1947)
Searching Possible Grasps... (Searched 25.68, Total Points: 1947)
Searching Possible Grasps... (Searched 30.82, Total Points: 1947)
Searching Possible Grasps... (Searched 35.95, Total Points: 1947)
Searching Possible Grasps... (Searched 41.09, Total Points: 1947)
Searching Possible Grasps... (Searched 46.22, Total Points: 1947)
Searching Possible Grasps... (Searched 51.36, Total Points: 1947)
Searching Possible Grasps... (Searched 56.50, Total Points: 1947)
Antipodal pairs found: [(549, 1589), (549, 1941), (552, 579), (552, 723), (552, 734), (552, 747), (552, 885), (552, 941), (552, 1022), (552, 1175), (552, 1189), (552, 1268), (552, 1370), (552, 1403), (552, 16

In [28]:
random_grasp_indices = random.choice(point_indices)
gripping_pos = (final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]) * .5
gripping_angle = final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]
gripping_angle /= np.linalg.norm(gripping_angle)

In [40]:
bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)
    
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.sync(C)

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 3., 0)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

bbox = final_points.get_axis_aligned_bounding_box()
min_bound = bbox.get_min_bound()
max_bound = bbox.get_max_bound()
center_point = (np.array(max_bound) + np.array(min_bound)) / 2.0

center_point += np.array([0., 0., .3])

komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1], center_point)
komo.addObjective([1.], ry.FS.vectorZ, ["l_gripper"], ry.OT.eq, [1e1], [0., 0., 1.])

komo.addObjective([2.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1], gripping_pos)
komo.addObjective([2.], ry.FS.vectorX, ["l_gripper"], ry.OT.eq, [1e1], gripping_angle)

ret = ry.NLP_Solver().setProblem(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=0).solve()
print(ret)
bot.move(komo.getPath(), [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

{ time: 0.006244, evals: 19, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0894211 }


In [ ]:
bot.home(C)
C.view(True)

0

In [41]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
